# <center><font color=teal>University Selection Modelling</font></center>

## <font color=teal>Initial Data Cleaning & EDA (Sprint 2)</font>

**BrainStation Bootcamp (12 September to 1 December 2023)**

*Capstone Project,     
Deadline: 30 November 2023   
Author: Reema Sipra*

---

##  <font color=teal>Table of Contents</font><a id = "TableOC"></a>

1. **[Introduction](#Intro)**
2. **[Notebook Environment](#NotebookEnv)**
3. **[Dataset Source](#DataSource)**
4. **[Consolidated data for Analysis](#ConsolidatedData)**
5. **[Next Steps](#NextSteps)**
6. **[References](#ReferenceList)**

---

##  <font color=teal>Introduction</font><a id = "Intro"></a>

One of the biggest decision points for students and their parents is picking a suitable university and career. People spend considerable resources to pick and get into the right university. With increasing educational costs and facing an increasingly competitive job market after graduation, making an informed decision is even more crtical for prospective students and their families.

The key to making an informed decision is relevant data that is presented in a useful manner. There is a wealth of information provided by govrenmental institutions and universities that can be analyzed through machine learning models to support such desicion making. Leveraging techniques such as time series analysis through AutoRegressive (AR) models and Convolutional Neural Networks (CNN), this project aims to provide a nuanced understanding of the complex interplay between these variables and future financial success.

Furthermore, these models have the potential to also support university with their strategies by enabling them to tailor their programs to better align with the financial expectations of students and enhance overall educational outcomes. It can also support diversification of the student body.

**Project Objectives:**
The objectives of the project have been updated to the following since Sprint-1:
- Support decision making of high school students & parents.
- Create a reliable model for students to estimate earnings for given program and university.

**Notebook (Sprint-2) Objectives:**
The objectives of this notebook are to:
- Create a dataframe with the necessary features for a bottom-up modelling approach.
- Setup the data for modelling.
- Perform preliminary EDA on the dataframe.

**Highlevel methodology**:
This will entail time series modelling of the projected earning based on selected key features. The key features will be determined through EDA and modelling iteratively through a bottom-up approach. For advanced modelling, Convolutional Nueral Networks can be considered as they are also used for time series analysis.

**Important notes on work progression** since *Initial Data Exploration (Sprint 1):*

Like any real world project, for this Capstone project there are updates, modifications, and revisits to the drawing board as the understanding of the problem space has grown. This is common in the early-stages of a project. It is important to front-load project modifications in the initial or conceptual stage of the project to clarify the scope and approah where the impact on the cost with respect to time and resources is considerably less compared to later stages (e.g. modelling, evaluation, deployment phases). The key modifications since Sprint-1:

1. **Title, Intent, and objective of the project**: The original intent was to predict enrollment of students into universities and this has been changed to a time series analysis of projected earnings e.g. for a given program / university etc combination.
2. **Methodology, EDA & modelling approach**: Originally models such as logistic regression, random forest, and neural networks were considered. Based on the changes in objectives, the approach has been altered in line with the 'Highlevel methodlogy' outlined earlier.
3. **Update of the Initial Data Exploration (sprint-1) notebook** inline with updates to the project.

[Back to the top](#TableOC)

## <font color=teal>Notebook Environtment</font><a id = "NotebookEnv"></a>

This project utilizes the "RIS_capstone_env" environment package created specifically for this project.

[Back to the top](#TableOC)

## <font color=teal>Dataset Source</font><a id = "DataSource"></a>

**Data Source:** 
US Department of Education, College Scorecard [[1]](#Ref1)

**College Scorecard:** An online tool created by the US government to allow users to compare the cost and value of higher institutional learning. Includes data on Title IV Universities (receive federal funding to make tuition affordable).

Initiated by President Obama to: " be able to see how much each school's graduates earn, how much debt they graduate with, and what percentage of a school's students can pay back their loans." 

**Three main components:** 
 - Institutional level data: (6543 universities, 15 categories, 3232 columns) over 26 years (across 26 csv files).
 - Fields of study at the universities (233,979 study areas, 160 columns) over 6 years (across 5 csv files).
 - Crosswalks:  map of differing institutional data defintion between university and the federal government (25004 records, 21 columns) over 20 years.



#### Data Dictionary:<a id = "DataDictionary"></a>

The dataset provided by College Score contains a wealth and abundance of well documented information of more than 6000 universities with 3000 columns. The following resources utilized together provide a good understanding of the dataset:
- A detailed glossary of terms can be found [here](https://surveys.nces.ed.gov/ipeds/public/glossary) [[2]](#Ref2).
- An excel file of the full dataset dictonary can be found [here](#https://collegescorecard.ed.gov/data/documentation/) [[3]](#Ref3).
- A data frame for the data dictionary of the institutional data was created for this project using the `yaml` file, included with the dataset [[1]](#Ref1). The data frame will be imported here from the `Initial Dataset Exploration` notebook and can be used to look up column information based on the `Column_id` from the institutional data frame from within this notebook.

In [1]:
# Retrieving the stored dataframe:
%store -r institution_column_data

In [2]:
# Check import was ok:
institution_column_data.set_index('Column_id', inplace=True)
institution_column_data.head(5)

,Category_level_1,Category_level_2,Category_level_3,column_description
Column_id,,,,
PCIP01,academics,program_percentage,agriculture,"Percentage of degrees awarded in Agriculture, ..."
PCIP03,academics,program_percentage,resources,Percentage of degrees awarded in Natural Resou...
PCIP04,academics,program_percentage,architecture,Percentage of degrees awarded in Architecture ...
PCIP05,academics,program_percentage,ethnic_cultural_gender,"Percentage of degrees awarded in Area, Ethnic,..."
PCIP09,academics,program_percentage,communication,Percentage of degrees awarded in Communication...


In [3]:
# Interactively search the `institutional_column_data` using this cell:

# Get the user input
search_name = input("Enter a column_id from the institutional dataset to search: ")

# Convert user input to match the case of the DataFrame index
search_name = search_name.upper()  

# Search the DataFrame based on user input
if search_name in institution_column_data.index:
    result = institution_column_data.loc[[search_name]]
    print(f"\nResult for {search_name}:\n{result}")
else:
    print(f"\nNo match found for {search_name}")

Enter a column_id from the institutional dataset to search:  PCIP09



Result for PCIP09:
          Category_level_1    Category_level_2 Category_level_3  \
Column_id                                                         
PCIP09           academics  program_percentage    communication   

                                          column_description  
Column_id                                                     
PCIP09     Percentage of degrees awarded in Communication...  


[Back to the top](#TableOC)

## <font color=teal>Consolidated data for Analysis</font><a id = "ReferenceList"></a>

#### Filtering the dataset:

1. **Number of Universities:** The dataset is quite extensive and includes 2-3 colleges and diploma /degree institutions along with institutions with 4-year and higher programs. The focus of this study is bachelor and graduate degrees. As such only institutions providing a highest degree (`HIGHDEG`) as bachelors or graduate degrees.

2. **Relevant to predict future earnings** Initially for the baseline this will be done as follows:

The main objective is to be able to use timeseries to make a prediction of the anticipated income a graduating student can expect once graduating from one of the title IV university within the United States (US).

Some of the **key parameters** we can have a look to predict the future earnings in the dataframe prepared for analysis are: 
- University id, ('UNITID')
- Universtiy name, ('INSTNM')
- Year ('Year')
- Location (latitude / longitude), ('LONGITUDE','LATITUDE')
- Region - the US states are divided into 10 regions ('REGION')
- Type of institution (Public, Private non-profit, Private for-profit), ('CONTROL')
- Highest degree awarded by institution ('HIGHDEG')
- Number of students, ('NUM4_PUB','NUM4_PRIV','NUM4_PROG')
- Enrollment of undergraduate certificate/degree-seeking students, ('UGDS')
- Enrollment of all undergraduate students, ('UG')
- Admission rate, ('ADM_RATE') 
- Average SAT equivalent score of students admitted, ('SAT_AVG') 
- 25th percentile of SAT scores at the institution (critical reading, math, writing), ('SATVR25','SATMT25','SATWR25')
- 75th percentile of SAT scores at the institution (critical reading, math, writing), ('SATVR75','SATMT75','SATWR75')
- Midpoint of SAT scores at the institution (critical reading, math, writing), ('SATVRMID','SATMTMID','SATWRMID')
- 25th percentile of ACT scores at the institution (cumulative, english, math, writing), ('ACTCM25','ACTEN25','ACTMT25','ACTWR25')
- 75th percentile of ACT scores at the institution (cumulative, english, math, writing), ('ACTCM75','ACTEN75','ACTMT75','ACTWR75')
- Midpoint of ACT scores at the institution (cumulative, english, math, writing), ('ACTCMMID','ACTENMID','ACTMTMID','ACTWRMID')
- Percentage of degrees awarded by program ( 'PCIP01,'PCIP03','PCIP04','PCIP05','PCIP09','PCIP10','PCIP11','PCIP12','PCIP13','PCIP14','PCIP15','PCIP16','PCIP19','PCIP22','PCIP23','PCIP24','PCIP25','PCIP26','PCIP27','PCIP29','PCIP30','PCIP31','PCIP38','PCIP39','PCIP40','PCIP41','PCIP42','PCIP43','PCIP44','PCIP45','PCIP46','PCIP47','PCIP48','PCIP49','PCIP50','PCIP51','PCIP52','PCIP54')
- Tuition / price, ('NPT4_PUB', 'NPT4_PRIV', 'NPT4_PROG') 
- Percentage of undergraduates who receive a Pell Grant, ('PCTPELL') 
- Percent of all undergraduate students receiving a federal student loan, ('PCTFLOAN')
- The median debt for students who have completed, ('GRAD_DEBT_MDN')
- The median debt for Pell students, ('PELL_DEBT_MDN')
- Share of students who received a federal loan while in school, ('LOAN_EVER')
- Share of students who received a Pell Grant while in school, ('PELL_EVER')
- Number of students not working and not enrolled 10 years after entry ('COUNT_NWNE_P10')
- Number of students working and not enrolled 10 years after entry('COUNT_WNE_P10')

Notes:
- (*) calculated fields of a running average of the previous 3 years.
- `Column_id` is given in brackets.

The **target variable** is:
- Potential earnings 10 years after entry to university (mean, median, 10, 25, 75, 90 percentile, standard deviation), ('MN_EARN_WNE_P10','MD_EARN_WNE_P10','PCT10_EARN_WNE_P10','PCT25_EARN_WNE_P10','PCT75_EARN_WNE_P10','PCT90_EARN_WNE_P10','SD_EARN_WNE_P10').

In [10]:
# Import the libraries:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
import re
import os
import warnings
warnings.filterwarnings("ignore")

First lets combine all the institual data across the years into a new dataframe. While each csv file is appended, add a new column providing the year.

In [8]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Create a consolidated instutituional dataframe

# Path to the institutional data csv files
csv_directory = './data'

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate through each CSV file in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith('.csv') and filename.startswith('MERGED'):
        # Construct the full file path
        file_path = os.path.join(csv_directory, filename)
        
        # Extract the year from the filename (assuming the year is part of the filename)
        year_part = filename.split('MERGED')[1].split('_')[0]  # Extract the part after 'MERGED' and before the underscore
        year = int(year_part)  # Convert the extracted part to an integer
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Add a new column 'Year' with the corresponding year
        df['Year'] = year
        
        # Append the current DataFrame to the list
        dataframes.append(df)
        
        # Combine all DataFrames in the list into one DataFrame
        combined_data = pd.concat(dataframes, ignore_index=True)

In [16]:
combined_data.tail()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMENRUP_NOPELL_FIRSTTIME,OMENRYP_NOPELL_NOTFIRSTTIME,OMENRAP_NOPELL_NOTFIRSTTIME,OMAWDP8_NOPELL_NOTFIRSTTIME,OMENRUP_NOPELL_NOTFIRSTTIME,OMACHT8_NOPELL_ALL,OMACHT8_NOPELL_FIRSTTIME,OMACHT8_NOPELL_NOTFIRSTTIME,ADDR,Year
183245,45896402,145954,1459.0,Strayer University-Charleston Campus,North Charleston,SC,29403,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010
183246,45897301,145990,1459.0,Strayer University-Irving,Irving,TX,75063,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010
183247,45897302,145992,1459.0,Strayer University-Katy,Houston,TX,77079,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010
183248,45897303,145994,1459.0,Strayer University-Northwest Houston,Houston,TX,77064-5768,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010
183249,45897304,145995,1459.0,Strayer University-Plano,Plano,TX,75093,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010


In [24]:
# Sanity check to make sure that all the years were appended.
sorted(combined_data['Year'].unique())

[1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

Now we need to first filter only the necessary columns to make the data more manageable for the first iteration of analysis. Since a bottom-up approach was adopted, other features can be included in further iterations for optimization.

In [25]:
# Create a list of the columns that we want to keep from the data frame:

column_list = ['UNITID','INSTNM','Year','LONGITUDE','LATITUDE','REGION','CONTROL','HIGHDEG',
               'NUM4_PUB','NUM4_PRIV','NUM4_PROG','UGDS','UG','ADM_RATE','SAT_AVG','SATVR25',
               'SATMT25','SATWR25','SATVR75','SATMT75','SATWR75','SATVRMID','SATMTMID','SATWRMID',
               'ACTCM25','ACTEN25','ACTMT25','ACTWR25','ACTCM75','ACTEN75','ACTMT75','ACTWR75',
               'ACTCMMID','ACTENMID','ACTMTMID','ACTWRMID','PCIP01','PCIP03','PCIP04','PCIP05','PCIP09',
               'PCIP10','PCIP11','PCIP12','PCIP13','PCIP14','PCIP15','PCIP16','PCIP19','PCIP22','PCIP23',
               'PCIP24','PCIP25','PCIP26','PCIP27','PCIP29','PCIP30','PCIP31','PCIP38','PCIP39','PCIP40',
               'PCIP41','PCIP42','PCIP43','PCIP44','PCIP45','PCIP46','PCIP47','PCIP48','PCIP49','PCIP50',
               'PCIP51','PCIP52','PCIP54','NPT4_PUB', 'NPT4_PRIV', 'NPT4_PROG','PCTPELL','PCTFLOAN',
               'GRAD_DEBT_MDN','PELL_DEBT_MDN','LOAN_EVER','PELL_EVER','COUNT_NWNE_P10','COUNT_WNE_P10',
               'MN_EARN_WNE_P10','MD_EARN_WNE_P10','PCT10_EARN_WNE_P10','PCT25_EARN_WNE_P10','PCT75_EARN_WNE_P10',
               'PCT90_EARN_WNE_P10','SD_EARN_WNE_P10']

In [31]:
# Create a new dataframe using the column list:
institutional_df = combined_data[column_list].copy()
institutional_df.head()

,UNITID,INSTNM,Year,LONGITUDE,LATITUDE,REGION,CONTROL,HIGHDEG,NUM4_PUB,NUM4_PRIV,NUM4_PROG,UGDS,UG,ADM_RATE,SAT_AVG,SATVR25,SATMT25,SATWR25,SATVR75,SATMT75,SATWR75,SATVRMID,SATMTMID,SATWRMID,ACTCM25,ACTEN25,ACTMT25,ACTWR25,ACTCM75,ACTEN75,ACTMT75,ACTWR75,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,PCIP12,PCIP13,PCIP14,PCIP15,PCIP16,PCIP19,PCIP22,PCIP23,PCIP24,PCIP25,PCIP26,PCIP27,PCIP29,PCIP30,PCIP31,PCIP38,PCIP39,PCIP40,PCIP41,PCIP42,PCIP43,PCIP44,PCIP45,PCIP46,PCIP47,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,NPT4_PUB,NPT4_PRIV,NPT4_PROG,PCTPELL,PCTFLOAN,GRAD_DEBT_MDN,PELL_DEBT_MDN,LOAN_EVER,PELL_EVER,COUNT_NWNE_P10,COUNT_WNE_P10,MN_EARN_WNE_P10,MD_EARN_WNE_P10,PCT10_EARN_WNE_P10,PCT25_EARN_WNE_P10,PCT75_EARN_WNE_P10,PCT90_EARN_WNE_P10,SD_EARN_WNE_P10
0,100654,Alabama A & M University,2011,NaN,NaN,5.0,1.0,4,644.0,NaN,NaN,4112.0,NaN,0.5010,846.0,370.0,360.0,370.0,450.0,450.0,440.0,410.0,405.0,405.0,16.0,14.0,15.0,NaN,19.0,19.0,18.0,NaN,18.0,17.0,17.0,NaN,0.0388,0.0145,0.0129,0.000,0.0000,0.0517,0.0258,0.0,0.1438,0.0856,0.0404,0.0000,0.0291,0.0,0.0162,0.0210,0.0,0.0985,0.0162,0.0,0.0,0.0000,0.0000,0.0000,0.0226,0.0,0.0630,0.0000,0.0646,0.0485,0.0,0.0,0.0,0.0,0.0162,0.0000,0.1906,0.0000,8734.0,NaN,NaN,0.6933,0.7903,26499,12092,0.9479553903,0.8756712102,214,1532,35300,31400,6700,16800,47000,70000,27800
1,100663,University of Alabama at Birmingham,2011,NaN,NaN,5.0,1.0,4,907.0,NaN,NaN,10854.0,NaN,0.7223,1107.0,500.0,500.0,NaN,630.0,640.0,NaN,565.0,570.0,NaN,21.0,21.0,20.0,NaN,27.0,28.0,26.0,NaN,24.0,25.0,23.0,NaN,0.0000,0.0000,0.0000,0.002,0.0427,0.0000,0.0139,0.0,0.0780,0.0551,0.0000,0.0124,0.0000,0.0,0.0223,0.0060,0.0,0.0819,0.0094,0.0,0.0,0.0000,0.0154,0.0000,0.0184,0.0,0.0641,0.0323,0.0149,0.0447,0.0,0.0,0.0,0.0,0.0417,0.2334,0.1822,0.0293,15013.0,NaN,NaN,0.3381,0.5355,17000,11500,0.9120045515,0.63967381,365,2661,46300,40300,10900,24900,57900,84000,41900
2,100690,Amridge University,2011,NaN,NaN,5.0,2.0,4,NaN,2.0,NaN,360.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0198,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.6931,0.0,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.2178,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0693,0.0000,NaN,4422.0,NaN,0.6506,0.7415,17740.5,6033,0.9389920424,0.8859416446,46,170,42100,38100,6800,20800,56100,79500,30500
3,100706,University of Alabama in Huntsville,2011,NaN,NaN,5.0,1.0,4,303.0,NaN,NaN,5755.0,NaN,0.6368,1164.0,500.0,520.0,NaN,630.0,670.0,NaN,565.0,595.0,NaN,22.0,22.0,21.0,NaN,29.0,30.0,28.0,NaN,26.0,26.0,25.0,NaN,0.0000,0.0000,0.0000,0.000,0.0224,0.0000,0.0292,0.0,0.0214,0.2471,0.0000,0.0224,0.0000,0.0,0.0204,0.0000,0.0,0.0652,0.0146,0.0,0.0,0.0000,0.0117,0.0000,0.0311,0.0,0.0331,0.0000,0.0000,0.0175,0.0,0.0,0.0,0.0,0.0360,0.1819,0.2325,0.0136,9677.0,NaN,NaN,0.3176,0.4779,20000,12750,0.8982905983,0.6717948718,178,1135,52700,46600,12900,29400,71300,93300,35900
4,100724,Alabama State University,2011,NaN,NaN,5.0,1.0,4,639.0,NaN,NaN,4719.0,NaN,0.5080,783.0,360.0,350.0,NaN,450.0,450.0,NaN,405.0,400.0,NaN,14.0,13.0,14.0,NaN,18.0,18.0,16.0,NaN,16.0,16.0,15.0,NaN,0.0000,0.0000,0.0000,0.000,0.0704,0.0000,0.0775,0.0,0.1778,0.0000,0.0000,0.0000,0.0000,0.0,0.0088,0.0000,0.0,0.1074,0.0123,0.0,0.0,0.0246,0.0000,0.0000,0.0053,0.0,0.0827,0.1056,0.0827,0.0211,0.0,0.0,0.0,0.0,0.0475,0.0335,0.1408,0.0018,5514.0,NaN,NaN,0.7251,0.7694,29002,9500,0.9238696809,0.9158909574,309,1846,30700,27800,7100,15700,42200,55400,21300


In [32]:
institutional_df.shape

(183250, 92)

In [34]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# describe numerical columns:
institutional_df.describe()

,UNITID,Year,LONGITUDE,LATITUDE,REGION,CONTROL,HIGHDEG,NUM4_PUB,NUM4_PRIV,NUM4_PROG,UGDS,UG,ADM_RATE,SAT_AVG,SATVR25,SATMT25,SATWR25,SATVR75,SATMT75,SATWR75,SATVRMID,SATMTMID,SATWRMID,ACTCM25,ACTEN25,ACTMT25,ACTWR25,ACTCM75,ACTEN75,ACTMT75,ACTWR75,ACTCMMID,ACTENMID,ACTMTMID,ACTWRMID,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,PCIP12,PCIP13,PCIP14,PCIP15,PCIP16,PCIP19,PCIP22,PCIP23,PCIP24,PCIP25,PCIP26,PCIP27,PCIP29,PCIP30,PCIP31,PCIP38,PCIP39,PCIP40,PCIP41,PCIP42,PCIP43,PCIP44,PCIP45,PCIP46,PCIP47,PCIP48,PCIP49,PCIP50,PCIP51,PCIP52,PCIP54,NPT4_PUB,NPT4_PRIV,NPT4_PROG,PCTPELL,PCTFLOAN
count,1.832500e+05,183250.000000,6046.000000,6046.000000,183249.000000,183221.000000,183250.000000,24226.000000,50583.000000,4781.000000,162227.000000,6304.000000,50821.000000,28709.000000,25602.000000,25738.000000,7037.000000,25603.000000,25736.000000,7037.000000,25596.000000,25732.000000,7037.000000,25705.000000,22326.000000,22316.000000,2087.000000,25700.000000,22324.000000,22313.000000,2086.000000,25692.000000,22317.000000,22307.000000,2086.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,167158.000000,24152.000000,50225.000000,4686.000000,91258.000000,84775.000000
mean,1.100459e+06,2008.634608,-90.529284,37.285374,4.562350,2.116766,2.206554,526.165731,148.855070,214.988078,2223.441832,2244.697335,0.695288,1073.191055,482.720334,485.005245,464.882052,590.375776,592.934450,571.341054,536.596578,539.012669,518.140827,20.174791,19.229956,19.077120,8.116435,25.366031,25.634832,24.977457,10.397411,23.028102,22.683918,22.286995,9.455896,0.004915,0.002681,0.001518,0.001280,0.012155,0.004558,0.034054,0.189087,0.027427,0.008944,0.026621,0.002852,0.007562,0.009991,0.008511,0.061453,0.000059,0.014604,0.002842,0.000106,0.008792,0.006599,0.005748,0.024756,0.004157,0.000633,0.017535,0.019959,0.006505,0.017915,0.009218,0.028879,0.009753,0.008448,0.034611,0.242970,0.117392,0.004703,9523.923319,18698.397232,19929.643833,0.486226,0.494703
std,5.113553e+06,7.397743,18.200380,5.869161,2.188437,0.837514,1.296166,640.185461,287.644607,504.635947,5039.702456,6422.655149,0.217306,134.111369,71.547208,75.118154,90.851890,69.026349,71.958204,95.064654,68.517932,71.901554,91.613820,3.641100,4.017796,3.540643,11.413032,3.520690,3.998449,3.579788,13.461082,3.502718,3.913122,3.471849,12.429551,0.036689,0.019088,0.024255,0.011318,0.052024,0.046258,0.096456,0.378356,0.073203,0.051689,0.098778,0.012881,0.032921,0.066483,0.029047,0.151241,0.001031,0.044649,0.008242,0.004717,0.041572,0.032537,0.056937,0.142980,0.018769,0.009375,0.049143,0.053725,0.027108,0.050371,0.050991,0.116638,0.055409,0.070216,0.134253,0.334291,0.175075,0.013125,4869.922148,7676.948514,9749.781964,0.230247,0.290090
min,1.006360e+05,1996.000000,-170.742774,-14.322636,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,514.000000,100.000000,100.000000,0.000000,100.000000,100.000000,0.000000,100.000000,100.000000,0.000000,1.000000,1.000000,1.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4837.000000,-103168.000000,-3552.000000,0.000000,0.000000
25%,1.644920e+05,2002.000000,-97.658609,33.932286,3.000000,1.000000,1.000000,112.000000,2

This dataframe has considerably fewer columns. The rows can now be filtered to make sure only undergraduate degrees are looked at.

[Back to the top](#TableOC)

## <font color=teal>Next Steps</font><a id = "NextSteps"></a>

- Further filter the institutional dataframe to only include bachelors degree.
- Finalise dataframe for analysis and modelling (cleaning, EDA, feature engineering for time series analysis). 
- Complete EDA.
- Finalize baseline (vanilla) model.

[Back to the top](#TableOC)

## <font color=teal>References</font><a id = "ReferenceList"></a>

<a id = "Ref1"></a>[[1]](#Data):  "*Data Home: Download the data*", v.Oct 2023, College Scorecard, US Dept. of Education, (https://collegescorecard.ed.gov/data/), last viewed: 10 Nov 2023.

<a id = "Ref1"></a> [[2]](#DataDictionary): "*View Glossary*", IPEDS 2023-24 Data Collection System, (https://surveys.nces.ed.gov/ipeds/public/glossary), last viewed: 10 Nov 2023.

<a id = "Ref1"></a> [[3]](#DataDictionary): "*Data Documentation*", v.Oct 2023, College Scorecard, US Dept. of Education, (https://collegescorecard.ed.gov/data/documentation/), last viewed: 10 Nov 2023.

[Back to the top](#TableOC)